In [15]:
import networkx as nx
import csv

G=nx.Graph()

node2id = {}
count = 0

file = open("string/9606.protein.actions.v11.0.txt")
reader = csv.reader(file, delimiter='\t')
firstline = True
for column in reader:
    if firstline:
        firstline = False
    else:
        n1 = -1
        n2 = -1
        if column[0] in node2id:
            n1 = node2id[column[0]]
        else:
            node2id[column[0]] = count
            n1 = count
            count += 1

        if column[1] in node2id:
            n2 = node2id[column[1]]
        else:
            node2id[column[1]] = count
            n2 = count
            count += 1

        G.add_node(n1)
        G.add_node(n2)
        score = int(column[6])
        if score>=700:
            G.add_edge(n1, n2)



In [18]:

file = open("string/9606.protein.aliases.v11.0.txt")
reader = csv.reader(file, delimiter='\t')
firstline = True
gene2protein = {}
for col in reader:
    if firstline:
        firstline = False
    else:
        prot = col[0]
        gene = col[1]
        if gene.find("ENSG")>-1 and prot in node2id:
            gene2protein[gene] = prot



In [4]:
import torch_geometric.utils as util

data = util.from_networkx(G)

degree = nx.degree(G)



In [5]:
data.edge_index

tensor([[    0,     0,     0,  ..., 16061, 16062, 16063],
        [    2,     3,     6,  ..., 13505,  5019,   853]])

In [6]:
data.num_nodes

16073

In [7]:
import torch

X = torch.ones(data.num_nodes, 1)
data.x = X

Y = torch.ones(data.num_nodes,1)
for i in range(0,data.num_nodes):
    Y[i,0] = degree[i]
data.y = Y

mask1 = torch.ones(data.num_nodes, dtype=bool)
mask2 = torch.ones(data.num_nodes, dtype=bool)
mask3 = torch.ones(data.num_nodes, dtype=bool)
for i in range(0,int(data.num_nodes*0.8)):
    mask1[i] = True
    mask2[i] = False
for i in range(int(data.num_nodes*0.8), data.num_nodes):
    mask1[i] = False
    mask2[i] = True
data.train_mask = mask1
data.test_mask = mask2
Y = Y / torch.max(Y)
data.y = Y
X = Y
#data.x = X


In [83]:
import gzip
import csv
import os

xs = []
count = 0
for i in [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("cancer/")) for f in fn]:
    if str(i).find("FPKM-UQ") > -1 and count < 2:
        count += 1
        file = gzip.open(i, mode="rt")
        csvobj = csv.reader(file, delimiter = '\t')
        localX = torch.zeros(data.num_nodes, 1)
        for line in csvobj:
            gene = line[0].split(".")[0]
            if gene in gene2protein:
                protein = gene2protein[gene]
                nodeid = node2id[protein]
                evalue = float(line[1])
                #print(gene + " is not missing: " + protein + " with id " + str(nodeid))
                localX[nodeid] = evalue
            else:
                continue
        xs.append(localX)
Xs = torch.stack(xs)
Xs = Xs / torch.max(Xs)
data.x = Xs
data.y = Xs



In [84]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(1, 32, improved=True)
        self.conv2 = GCNConv(32, 1, improved=True)

    def forward(self, data): # should really be trained in batches somehow, probably a terrible hack
        xlist, edge_index = data.x, data.edge_index
        out = []
        for i in range(0, len(xlist)):
            x = xlist[i]
            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
            x = self.conv2(x, edge_index)
            out.append(torch.sigmoid(x))
        return torch.stack(out)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data_run = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    out = model(data_run)
    #### SUM UP HERE
    loss = float(0.0)
    for i in range(0,len(data_run.x)):
        localloss= F.mse_loss(out[i][data_run.train_mask], data_run.y[i][data_run.train_mask])
        loss += localloss
    if epoch % 10 == 0:
        print(epoch, loss)
    loss.backward()
    optimizer.step()

    

0 tensor(0.4942, grad_fn=<AddBackward0>)


10 tensor(0.3639, grad_fn=<AddBackward0>)


20 tensor(0.2198, grad_fn=<AddBackward0>)


30 tensor(0.1066, grad_fn=<AddBackward0>)


40 tensor(0.0467, grad_fn=<AddBackward0>)


50 tensor(0.0215, grad_fn=<AddBackward0>)


60 tensor(0.0107, grad_fn=<AddBackward0>)


70 tensor(0.0057, grad_fn=<AddBackward0>)


80 tensor(0.0035, grad_fn=<AddBackward0>)


90 tensor(0.0024, grad_fn=<AddBackward0>)
